In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_5101/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_5101/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_6_46_2,0.952057,0.838913,0.868917,0.974760,0.349908,1.077188,1.034321,0.232992,0.810369,0.591530,0.938590,0.596742,52.100170,120.723473,"Hidden Size=[4, 2], regularizer=0.02, learning..."
1,model_6_45_7,0.952148,0.837344,0.864837,0.961095,0.349246,1.087684,1.066519,0.212847,0.803121,0.590970,0.938707,0.596177,52.103959,120.727262,"Hidden Size=[4, 2], regularizer=0.02, learning..."
2,model_6_46_7,0.951587,0.836476,0.873070,0.969449,0.353344,1.093488,1.001553,0.282017,0.811987,0.594427,0.937987,0.599665,52.080626,120.703930,"Hidden Size=[4, 2], regularizer=0.02, learning..."
3,model_6_45_3,0.952031,0.836195,0.865724,0.965028,0.350101,1.095364,1.059517,0.191328,0.814296,0.591693,0.938557,0.596906,52.099069,120.722372,"Hidden Size=[4, 2], regularizer=0.02, learning..."
4,model_6_45_0,0.952011,0.836076,0.865556,0.965420,0.350243,1.096159,1.060840,0.189188,0.812096,0.591813,0.938532,0.597028,52.098255,120.721558,"Hidden Size=[4, 2], regularizer=0.02, learning..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,model_4_6_6,0.969510,-0.936305,-2.016196,0.875606,0.222529,12.948078,23.799587,0.659632,0.393606,0.471730,0.795539,0.475886,197.005398,463.263814,"Hidden Size=[24], regularizer=0.02, learning_r..."
641,model_4_7_7,0.980414,-1.089068,-2.065341,0.966504,0.142946,13.969610,24.187370,0.349572,0.463213,0.378083,0.868660,0.381414,197.890570,464.148986,"Hidden Size=[24], regularizer=0.02, learning_r..."
642,model_4_7_2,0.978219,-1.284402,-1.963289,0.965691,0.158965,15.275807,23.382119,0.358051,0.339677,0.398704,0.853942,0.402216,197.678148,463.936565,"Hidden Size=[24], regularizer=0.02, learning_r..."
643,model_4_6_9,0.963119,-1.286784,-2.108502,0.821088,0.269177,15.291733,24.527936,0.948727,0.416472,0.518822,0.752679,0.523394,196.624774,462.883190,"Hidden Size=[24], regularizer=0.02, learning_r..."
